<a href="https://colab.research.google.com/github/QwuophyRain/Heart-Failure-Prediction-Flask-App-project/blob/main/HeartFailureApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()


Saving heart_failure_clinical_records_dataset (1).csv to heart_failure_clinical_records_dataset (1).csv


In [3]:
!pip install joblib


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib

# Load the dataset
df = pd.read_csv("heart_failure_data.csv")

# Check column names
print(df.columns)

# Define input features and target (update 'target' if different)
X = df.drop("DEATH_EVENT", axis=1)
y = df["DEATH_EVENT"]

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=1)
model.fit(X_train, y_train)

# Evaluate model
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy*100:.2f}%")

# Save the trained model to file
joblib.dump(model, "model.pkl")

Index(['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes',
       'ejection_fraction', 'high_blood_pressure', 'platelets',
       'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time',
       'DEATH_EVENT'],
      dtype='object')
Accuracy: 93.33%


['model.pkl']

In [7]:
files.download("model.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
from flask import Flask, render_template, request
import joblib
import numpy as np

# Initialize the Flask app
app = Flask(__name__)
model = joblib.load('model.pkl')  # Load the trained model

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Extract values from the form and convert to float
        inputs = [float(x) for x in request.form.values()]
        prediction = model.predict([inputs])[0]

        message = "Heart Disease Detected" if prediction == 1 else "No Heart Disease Detected"
        return render_template('index.html', prediction_text=message)
    except Exception as err:
        return render_template('index.html', prediction_text=f"Error: {err}")

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [9]:
!pip install flask-ngrok
!pip install joblib


In [ ]:
from flask import Flask, request, render_template_string
from flask_ngrok import run_with_ngrok
import joblib
import numpy as np

app = Flask(__name__)
run_with_ngrok(app)  # expose app via ngrok

model = joblib.load('model.pkl')  # Make sure this is uploaded

@app.route('/')
def home():
    return '''
        <form action="/predict" method="post">
            <input type="text" name="age" placeholder="Age"><br>
            <input type="text" name="sex" placeholder="Sex"><br>
            ...
            <input type="submit">
        </form>
    '''

@app.route('/predict', methods=['POST'])
def predict():
    try:
        features = [float(x) for x in request.form.values()]
        pred = model.predict([features])[0]
        result = "Heart Disease Detected" if pred == 1 else "No Heart Disease"
        return f"<h1>{result}</h1>"
    except Exception as e:
        return f"<h1>Error: {e}</h1>"

app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-9:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
            